# Project Title

NLTK ile Twitter üzerinde duygu analizi 

## Members

Betül Durkaya, durkaya@itu.edu.tr

## Description of the project

      DUYGU ANALİZİ(SENTIMENT ANALYSIS)

    Duygu analizi(sentiment analysis), fikir madenciliği(opinion mining) olarak da bilinir; insanların hizmetler, kuruluşlar, bireyler, konular, etkinlikler varlıklar ve ürünlere yönelik duygularını, fikirlerini, değerlendirmelerini ve tutumlarını analiz eden bir çalışma alanıdır. Bu büyük bir sorun alanı temsil eder. Ayrıca duygu analizi(sentiment analysis), görüş incelemesi(opinion mining), fikir çıkarımı(opinion extraction), duygu madenciliği(sentiment mining), öznel analiz(subjectivity analysis), etki analizi(affect analysis), inceleme madenciliği(rewiew mining) gibi birçok isim ve biraz farklı görevler de vardır. Ancak, şimdi hepsi duyarlılık analizi(sentiment analysis) ya da fikir madenciliği(opinion mining) çatısı altında toplanmaktadır. Endüstride iken, duygu analizi terimi daha yaygın olarak kullanılmaktadır; akademik çevrelerde hem duyarlılık analizi hem de görüş incelemesi terimi sıklıkla kullanılmaktadır.Ne olursa olsun, temelde aynı çalışma alanını temsil ediyorlar. Ancak, bazı araştırmacılar duygu analizi ve fikir madenciliğinin biraz farklı görüşleri olduğunu belirttiler. Fikir madenciliği; insanın bir varlık hakkındaki fikirlerini ayıklar ve analiz eder. Duygu analizi ise bir metinde ifade edilen duyguyu tanımlarken daha sonra bunu analiz eder. Dolayısıyla, duygu analizinin hedefi, görüşleri bulmak, ifade ettikleri duyguları belirlemek ve daha sonra kutuplarını sınıflandırmaktır. Duyarlılık analizi, çoğunlukla olumlu veya olumsuz düşünceleri ifade eden veya ima eden görüşlere odaklanır. Bir fikrin duygusal olarak hangi sınıfa girdiğini belirlemeyi amaçlar. Bu sınıflar olumlu(positive), olumsuz(negative) ve nötr(neutral) olarak adlandırılabilir.
   
   Dilbilimin ve doğal dil işlemenin(NLP-Natural Language Process) uzun bir geçmişi olmasına rağmen, 2000 yılından önce insanların görüş ve düşünceleri konusunda çok az araştırma yapılmıştır. O günden bu yana, alan çok aktif bir araştırma alanı haline gelmiştir. Bunun birkaç nedeni var: Birincisi, neredeyse her alanda geniş bir uygulama dizisi vardır. Duyarlılık analizini çevreleyen endüstri de ticari uygulamaların çoğalmasıyla gelişti. Bu araştırma için güçlü bir motivasyon sağlar. İkincisi, daha önce hiç araştırılmamış olan birçok zorlu araştırma problemi sunuyor. Üçüncüsü, insanlık tarihinde ilk kez, Web'deki sosyal medyada çok sayıda fikre sahip veri mevcut. Bu veriler olmadan, bir sürü araştırma mümkün olmazdı. Şaşırtıcı olmayan bir şekilde, duygu analizinin başlangıcı ve hızlı bir şekilde büyümesi, sosyal medyanın gelişimi ile çakışmaktadır. Aslında, duygu analizi şu anda sosyal medya araştırmasının merkezinde Bu nedenle, duyarlılık analizi araştırması yalnızca NLP üzerinde önemli bir etkiye sahip olmakla kalmaksızın aynı zamanda yönetim bilimleri, siyaset bilimi, ekonomi ve sosyal bilimler üzerinde de etkiler yapabilir, çünkü hepsi insanların görüşlerinden etkilenir. Duygu analizi araştırması ağırlıklı olarak 2000 yılı başından itibaren başlamış olmasına rağmen, duygu sıfatları, öznellik, bakış açısı ve etkileri üzerine daha önce yapılmış bazı çalışmalar vardır.


Duygu Analizi Uygulamaları(Sentiment Analysis Applications)

   Fikirler neredeyse tüm insan faaliyetlerinin merkezinde yer alırlar çünkü davranışlarımızın önemli etkeni bunlardır. Ne zaman bir karar vermemiz gerekse başkalarının görüşlerini bilmek isteriz. Gerçek dünyada, işletmeler ve kuruluşlar her  zaman ürün ve hizmetleri hakkında tüketici veya halkın fikirlerini bulmak istemektedir. Tüketiciler, bir ürünün satın almadan önce mevcut kullanıcılarının görüşlerini veya siyasi seçimler için bir oy vermeden önce başkalarının siyasi adaylarla ilgili görüşlerini bilmek istemektedir. Geçmişte, bir kişinin görüşe ihtiyacı olduğunda arkadaşları ve ailesine danışırdı. Bir kuruluşun veya bir işletmenin halk veya tüketici görüşlerine ihtiyacı olduğunda incelemeler ve anketler gerçekleştirirdi. Halk ve tüketici görüşlerini edinmek uzun zamandır pazarlama, halkla ilişkiler ve siyasi kampanya şirketleri için büyük bir ticari iştir. 

   Web'de sosyal medyanın (örneğin incelemeler, forum tartışmaları, bloglar, mikrobloglar, Twitter, yorumlar ve sosyal sitelerdeki yayınlar) büyümesiyle, bireyler ve kuruluşlar karar vermede bu medyadaki içeriği giderek kullanıyor. Günümüzde bir kişi bir ürün satın almak istiyorsa, bu kişi sadece arkadaşları ve ailesinden fikir istemekle sınırlı değildir, çünkü ürünle ilgili Web'de kamuya açık forumlarda çok sayıda kullanıcı görüşleri bulunmaktadır. Bir organizasyon açısından; halkın fikirlerini toplamak için anketler yapmak artık gerekli değildir. Çünkü böyle bir bilgi kamuya açık bir şekilde mevcuttur. Ancak, çeşitli sitelerin çoğalması yüzünden Web'deki görüş sitelerini bulma ve bunlarda yer alan bilgileri dağıtma zorlu bir görev olmayı sürdürüyor. Her site genellikle uzun bloglarda ve forum ilanlarında kolaylıkla çözülemeyen büyük bir fikir metni içeriyor. Ortalama bir okuyucu, ilgili siteleri belirlemekte ve bunlardaki fikirleri özetlemekte zorlanacaktır. Bu nedenle otomatik duyarlılık analiz sistemlerine ihtiyaç duyulmaktadır. 

   Son yıllarda, toplumsal ve siyasal sistemler üzerinde derin etkilere sahip olan sosyal medyada görüşlü mesajların işletmelerin ve halk duygularının yeniden şekillenmesine yardımcı olduğunu gördük. Bu tür ilanlar, 2011'de bazı Arap ülkelerinde yaşananlar gibi siyasi değişim için kitleleri harekete geçirdi. Dolayısıyla, Web üzerinde fikir toplamak ve incelemek bir zorunluluk haline geldi. Elbette, fikir verilen belgeler yalnızca Web'de varolmakla kalmaz, aynı zamanda birçok kurumda dahili veriler bulunur. Örneğin; e-postalardan ve çağrı merkezlerinden toplanan müşteri geri bildirimleri veya kuruluşlar tarafından yapılan anketlerden alınan sonuçlar. 

   Bu uygulamalar nedeniyle son yıllarda sanayi faaliyetleri gelişti. Duyarlılık analizi uygulamaları, tüketici ürünleri, servisler, sağlık hizmetleri ve sosyal etkinliklere finansal hizmetler ve siyasi seçimler arasında neredeyse her alana yayılmıştır. Microsoft, Google, Hewlett-Packard, SAP ve SAS gibi birçok büyük şirket, kendi kurum içi yeteneklerini inşa etmiştir. Bu pratik uygulamalar ve endüstriyel çıkarlar, duygu analizi araştırmaları için güçlü motivasyonlar sağlamıştır. Gerçek hayatta olan uygulamaların yanı sıra, birçok uygulamaya yönelik araştırma makaleleri de yayınlanmıştır.

Duygu Analizi Araştırması(Sentiment Analysis Research)

   Gerçek hayatta uygulamalar, duyarlılık analizinin popüler bir araştırma problemi haline gelmesinin yalnızca bir parçası. Bir NLP araştırma konusu olarak da son derece zorlayıcıdır. Ek olarak, 2000 yılından önce NLP'de veya dilbilimde çok az araştırma yapıldı. Bunun nedenlerinden biri, o zamandan beri sayısal formlarda az sayıda görüş metninin mevcut olmasıdır. 2000 yılından beri bu alan, NLP'nin en aktif araştırma alanlarından biri haline geldi. Ayrıca, veri madenciliği, Web madenciliği ve bilgi edinme konularında geniş çapta araştırılmaktadır. Aslında, bilgisayar bilimlerinden yönetim bilimlerine yayılmıştır.

Analizin Farklı Seviyeleri(Different Levels of Analysis)

   Product Reviews -> Sentiment Identification -> Feature Selection -> Sentiment Classification -> Sentiment Polarity
   
   Duygu analizi bir sınıflandırma işlemi olarak düşünülebilir. Temel olarak üç düzeyde incelenmiştir:

   Doküman seviyesi(Document level):Bu seviyedeki görev, bir düşünce belgesinin olumlu veya olumsuz bir düşünceyi ifade edip etmediğini sınıflandırmaktır. Örneğin, bir ürün incelemesi göz önüne alındığında; sistem, incelemenin ürünle ilgili genel bir olumlu veya olumsuz görüş bildirip bildirmediğini belirler. Bu görev yaygın olarak belge seviyesinde(document level) duyarlılık sınıflandırması(sentiment classification) olarak bilinir. Bu analiz düzeyi, her belgenin tek bir varlık üstünde görüş bildirdiğini varsayar. Bu nedenle, birden çok varlığı değerlendiren veya karşılaştıran belgeler için geçerli değildir.

   Cümle seviyesi(Sentence level):Bu seviyedeki görev, her cümlenin olumlu(positive), olumsuz(negative) veya nötr(neutral) görüşlerini ifade edip etmediğini belirler. Nötr genellikle bir görüş içermez. Bu analiz seviyesi, öznel düşünceleri ifade eden cümlelerle nesnel bilgi ifade eden cümleleri ayıran öznellik sınıflandırmasıyla(subjectivity classification) yakından ilişkilidir. Doküman seviyesinde analizden büyük bir farkı yoktur, çünkü cümlerler de kısa dokümanlardır.

   Varlık ve yön düzeyi(Entity and aspect level): Hem belge düzeyi hem de cümle düzeyinde yapılan analizler tam olarak insanların ne beğendikleri ve beğenmediğini bulmazlar. Yön düzeyi daha ayrıntılı analiz yapar. Yön düzeyi daha önce özellik seviyesi olarak adlandırılmıştır. Dil yapılarına bakmak yerine, yön düzeyi doğrudan görüşe bakar. Görüş sahipleri, aynı varlıkta farklı yönlerden farklı fikirler verebilirler. Örneğin; “Although the service is not that great, I still love this restaurant." Cümle restaurant hakkında pozitif ancak servis hakkında negatiftir.
   
   Duyarlılık Sınıflandırma Teknikleri(Sentiment Classification Techniques)
   
   Duyarlılık Sınıflandırma teknikleri; makine öğrenme yaklaşımı(the machine learning approach), sözlüğe dayalı yaklaşım(lexicon based approach) ve hibrid yaklaşım(hybrid approach) olarak ayrılır. Makine Öğrenme Yaklaşımı, ünlü machine learning algoritmalarını uygular ve dilsel özellikleri kullanır. Sözlük tabanlı yaklaşım, bilinen ve önceden derlenmiş duygu terimlerinin bir koleksiyonu olan bir duygu sözlüğüne dayanır. Melez yaklaşım, her iki yaklaşımı birleştirir.
 
   Machine learning ile sınıflandırma, denetimli ve denetimsiz öğrenme yöntemlerine bölünebilir. Denetlenen yöntemler, çok sayıda etiketli eğitim belgelerinden yararlanmaktadır. Bu etiketli eğitim belgelerinin bulunması güç olduğunda denetimsiz yöntemler kullanılır.
 
   Lexicon yaklaşımı, metnin analizinde kullanılan görüş sözlüğünün bulunmasına bağlıdır. Bu yaklaşımda iki yöntem vardır: Sözlük tabanlı yaklaşım(dictionary-based approach); kök sözcüklerini bulmaya dayanır,eşanlamlılarının ve zıt anlamlılarının sözlüğünü araştırır. Korpus tabanlı yaklaşım(corpus-based approach); görüş kelimelerinin bir kök listesi ile başlar ve diğer görüş kelimelerini büyük bir korpus içerisinde bulur. Bu, istatistiksel veya semantik yöntemler kullanılarak yapılabilir.
 
   Duygu Sözlüğü ve Sorunları(Sentiment Lexicon and Its Issues)

   Şaşırtıcı olmayan bir şekilde, duyguların en önemli göstergeleri, fikir sözcükleri olarak da bilinen duygu sözcükleridir. Bunlar, pozitif veya negatif duyguları ifade etmek için sıklıkla kullanılan kelimelerdir. Örneğin; iyi, harika ve şaşırtıcı olumlu düşünce kelimeleridir ve kötü, yoksul ve korkunç olumsuz duygu sözcükleridir. Kelimelerin yanı sıra, cümle ve deyimler de vardır; duyarlılık sözcükleri ve deyimler açık nedenlerden ötürü duygu analizine aracı olur. Bu sözcük ve deyimlerin bir listesine bir duygu sözlüğü denir. Araştırmacılar, yıllar geçtikçe bu sözlükleri derlemek için çok sayıda algoritma tasarladılar. 
   
   Duygu kelimeleri ve cümleleri duygu analizi için önemli olmasına rağmen, bunları kullanmak yeteri kadar değildir. Sorun çok daha karmaşıktır. Başka bir deyişle, duygu sözlüğünün duygu analizi için gerekli olduğu ancak yeterli olmadığı söylenebilir. Olumlu veya olumsuz bir duygu kelimesi, farklı uygulama alanlarında zıt yönlere sahip olabilir. Örneğin; "suck" genellikle olumsuz düşünceyi gösterir. “This camera sucks.” örneğindeki gibi. Fakat pozitif görüşte bildirebilir. Örneğin “This vacuum cleaner really sucks.”  

   Twitter Üzerinde Duygu Analizi(Sentiment Analysis on Twitter)
   
   Sosyal medya platformları, farklı konularda düşüncelerini ifade etmek için farklı kişiler tarafından kullanılır, bu nedenle de insanların görüşlerinin değerli bir kaynağıdır. Twitter, kullanıcıların oluşturduğu çok sayıda metin mesajı içerir ve her geçen gün büyür. Twitter'ın izleyici kitleleri sıradan kullanıcılardan ünlülere, şirket temsilcilerine, siyasetçilere ve hatta ülke başkanlarına kadar değişir. Bu nedenle, farklı sosyal vgurplardan kullanıcıların metin mesajlarını toplamak mümkündür. Mesajların içeriği kişisel düşüncelerden kamuya açıklamalara kadar değişir.

   Dünyanın herhangi bir yerinden veri edinebilirsiniz. Twitter'ın kitlesi birçok ülkeden kullanıcılar tarafından temsil edilmektedir. ABD'li kullanıcılar hakim olsa da, farklı dillerde veri toplamak mümkündür. 

   Twitter API'si iyi tasarlanmış ve kolay erişilebilirdir. Twitter verilerini analiz için uygun bir biçimdedir. Twitter'ın kullanım şartları, diğer API'lere kıyasla nispeten serbesttir. Bu özelliklerden ötürü duygu analizi için Twitter'ı kullandık.

   Twitter API(Application Programming Interface)
   
   Uygulama Programlama Arayüzü(API), bir web tabanlı yazılım uygulamasına erişmek için kullanılan bir dizi programlama standardıdır. Twitter, Reprsentational State Transfer(REST) mimarisinin API'sini temel alır. REST mimarisi, kaynakları tanımlayan ve veriye erişim yollarını belirleyen ağ tasarım ilkelerinin bir koleksiyonunu ifade eder.


   NLTK(NATURAL LANGUAGE TOOLKIT)

NLTK(natural language toolkit), Python'da doğal dil işleme(natural language processing) için kütüphaneler ve programlar içeren bir araçtır. Başlangıçta Steven Bird, Edward Loper ve Ewan Klein tarafından geliştirilmiştir. Tokenization, ayrıştırma(parsing), sınıflandırma(classification), köklendirme(stemming), etiketleme(tagging), anlamsal akıl yürütme(semantic reasoning) için metin işleme(text processing) kütüphaneleri içerir.

Referanslar(References)

http://www.morganclaypool.com/doi/pdfplus/10.2200/S00416ED1V01Y201204HLT016
http://www.sciencedirect.com/science/article/pii/S2090447914000550
http://crowdsourcing-class.org/assignments/downloads/pak-paroubek.pdf
https://www.irjet.net/archives/V4/i3/IRJET-V4I3581.pdf

### The methods to be used

Proje için Python programlama dili kullanılacak. Twittera erişim için bazı anahtarları(consumer key, consumer secret, access token ve access token secret) elde etmemiz gerekiyor, bunu Twitter API yardımıyla yapacağız. Twitter'a Python'la yazdığımız kodlarla istekte bulunacağız ve bağlantı tweepy kütüphanesi ve erişim anahtarlarımız sayesinde gerçekleşecek. Ek olarak json kütüphanesini kullanacağız, bu yüzden Twitter'dan dönen cevap json formatında olacak. Başlangıçta json datası bir txt dosyasında biriktirilecek, sonrasında datayı bir databasede saklayacağız. Bundan sonra stringleri istenen bileşenlere ayıracağız. Stringlerin birer parçası olan kelimeleri, cümlecikleri ve sembolleri "token" olarak isimlendrilir, ayrıca bu işleme tokenization ya da tokenizing denir. Ardından stemming ve lemmatization işlemlerini yapacağız. Stemming; sözcüğün kökünü bulmak için kelime son eklerini değiştirme ve kaldırma yöntemidir. Lemmatization ise farklılaşmış sözcük biçimlerini bir araya getirme işlemidir. Sonra, verileri NLTK ve TextBlob kullanarak analiz edeceğiz.

### The data

Twitter üzerinden tweetler data olarak kullanılacak ve tweetler json formatında toplanacak.

## Code


In [1]:
import tweepy
import json
from processes import tokenize_process, sample_analysis


# organizing tweet information
class StreamListener(tweepy.StreamListener):
    def on_connect(self):
        print("Tweet streaming begin.")

    def on_error(self, status):
        print('Error Type: ' + status)
        return False

    def on_data(self, raw):
        data = json.loads(raw)
        tweet_id = data['id_str']  # The ID of tweet from Twitter in string format
        time = data['created_at']  # The time of creation of the tweet
        username = data['user']['screen_name']  # The Tweet author's username
        text = data['text']  # The entire body of the Tweet
        fs = open("tweets.txt", "a")

        try:
            # insert tweet data to tweet.txt file if RT is not exist
            if data['text'].find('RT @') is -1:
                fs.write(tweet_id + '\t' + time + '\t' + username + '\n' + text + '\n\n\n')
                print(tweet_id + '\t' + time + '\t' + username + '\n' + text)
                tokenize_process(tweet_id, text)    # tokenize tweets
                sample_analysis(text)   # Analysis example
            fs.close()
        except Exception as e:
            print(e)
        return True


In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from textblob import Word, TextBlob


def lemmatize_process(word):
    w = Word(word)
    w.lemmatize()
    return w


def stem_process(word):
    grouping = PorterStemmer()
    grouping.stem(word)


def tokenize_process(tweet_id, tweet):
    fs = open("tokenize.txt", "a")
    lemmatized = []
    tokens = word_tokenize(tweet)
    for index in range(len(tokens)):
        lemmatized.append(lemmatize_process(tokens[index]))

    fs.write('id: ' + tweet_id + '{ ')
    fs.write(str(lemmatized))
    fs.write(' }\n\n')
    print(lemmatized)
    fs.close()


def sample_analysis(tweet):
    sample = TextBlob(tweet)
    print(sample.sentiment, '\n')


In [3]:
import tweepy
from tweepy import OAuthHandler
import listening

#twitter api keys to authanticate
consumer_key = "dtKTW35ISxLtPDCszilhNk6R2"
consumer_secret = "pDoisuGvzTGYte5rNEtjguiixVyys0IrL98xu5tzxE7aG8KZCJ"

access_token = "773843946-LBFza4JmUOrzrUOkahDdtHXi6AHwFQyjwH6j7Ibg"
access_token_secret = "JU0ExxNVFkQbBzipW0EOL9Lkb6dm3RadGdfoJeaLdOUgm"

#authentication sections
auth = OAuthHandler(consumer_key, consumer_secret)
print("First auth done")
auth.set_access_token(access_token, access_token_secret)
print("2nd auth done")

#tracked word
WORDS = ['iphone']

#from listening library, StreamListener object created
listener = listening.StreamListener(api=tweepy.API(wait_on_rate_limit=True))
#from tweepy library, Stream begins
streamer = tweepy.Stream(auth=auth, listener=listener)

print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS, languages=["en"]) #filtering keywords

First auth done
2nd auth done
Tracking: ['iphone']
Tweet streaming begin.
853675534395609088	Sun Apr 16 18:24:42 +0000 2017	SolisFarid
Apple iPhone 6 Factory Unlocked 16GB Smartphone FAIR CONDITION Giveaway RT &amp; Follow https://t.co/iLLoEr1Guy
['Apple', 'iPhone', '6', 'Factory', 'Unlocked', '16GB', 'Smartphone', 'FAIR', 'CONDITION', 'Giveaway', 'RT', '&', 'amp', ';', 'Follow', 'https', ':', '//t.co/iLLoEr1Guy']
Sentiment(polarity=0.7, subjectivity=0.9) 

853675565102116864	Sun Apr 16 18:24:49 +0000 2017	DesSays_
Which? I have a 5 lo, https://t.co/ZY0J5rfZdR
['Which', '?', 'I', 'have', 'a', '5', 'lo', ',', 'https', ':', '//t.co/ZY0J5rfZdR']
Sentiment(polarity=0.0, subjectivity=0.0) 

853675597708611585	Sun Apr 16 18:24:57 +0000 2017	ayoo_tezz
Check out this iPhone/iPad game I'm playing called Basketball Agent. https://t.co/D1qAQf7zFR https://t.co/S6e8Lfulhi
['Check', 'out', 'this', 'iPhone/iPad', 'game', 'I', "'m", 'playing', 'called', 'Basketball', 'Agent', '.', 'https', ':', '//t.c

KeyboardInterrupt: 